In [0]:
!pip install tensorflow_graphics

import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub

In [0]:
print(tf.__version__)

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))




In [0]:
#SDR_path = tf.keras.utils.get_file('sdrlake.jpg', 'sdrlake.jpg')
  

#HDR_path = tf.keras.utils.get_file('hdrlake.jpg','https://drive.google.com/open?id=1o7STsmUqZqI0--YygR0pEi0NOyROB0Ah')
  
#https://storage.googleapis.com/download.tensorflow.org/example_images/Green_Sea_Turtle_grazing_seagrass.jpg
#1hTZW51H8M7t1_xW0xYo1kq9Osq_EsrmQ
    
def load_img(path_to_img):
  max_dim = 3840
  
  img = tf.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  
  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  
  return img

#def imshow(image, title=None):
#  plt.imshow(image)
#  if title:
#    plt.title(title)

image_tensor = tf.image.decode_jpeg(tf.read_file("sample_data/SDRLake.jpg"), channels=3)
    
SDR_image = load_img("sample_data/SDRLake.jpg")
HDR_image = load_img("sample_data/HDRLake.jpg")

x = tf.keras.applications.vgg19.preprocess_input(SDR_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape

#plt.subplot(1, 2, 1)
#imshow(SDR_image, 'Content Image')

#plt.subplot(1, 2, 2)
#imshow(HDR_image, 'Style Image')    
    
    

#image_tensor = tf.image.decode_jpeg(tf.read_file("SDRLake.jpg"), channels=3)
#sess = tf.InteractiveSession()

#tfgray = tf.image.rgb_to_grayscale(image)

#gray_encoded = tf.image.encode_jpeg(tfgray,name="save_me")

#fwrite = tf.write_file("grayLake.jpg", gray_encoded)

#img16 = (65536/image).astype('uint16')

#hub.get_expected_image_size(image)

#img16 = np.int16(image)     # convert to signed 16 bit integer to allow overflow
#img16 = (65536/255)*img16  # apply scale factor

#encoded16 = tf.image.encode_jpeg(img16,name="save_me")

#fwrite = tf.write_file("Lake16.jpg", encoded16)

#x = tf.constant([[1.0, 0.0], [0.0, 1.0]])
#image_tensor = tf.math.multiply(image_tensor, 2)

#sess = tf.Session()
#print sess.run(y)
# ==> [[2.0, 0.0], [0.0, 2.0]]

#output = image_tensor[0, :]
#print sess.run(output) 

#color_encoded = tf.image.encode_jpeg(image_tensor,name="save_me")

#fwrite = tf.write_file("colorLake.jpg", color_encoded)

#result = sess.run(image_tensor)

#sess.run(fwrite)

#print(result[1920, 1080])

# Add print operation
#a = tf.Print(image_tensor, [image_tensor], message="This is a: ")

# Add more elements of the graph using a
#b = tf.add(a, a)



# New Section

In [0]:
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer of interest
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

In [0]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer of interest
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(HDR_image*255)

#Look at the statistics of each layer's output
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

In [0]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}
  
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(SDR_image))

style_results = results['style']

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())

In [0]:
style_targets = extractor(HDR_image)['style']
content_targets = extractor(SDR_image)['content']

image = tf.Variable(SDR_image)

def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

opt = tf.train.AdamOptimizer(learning_rate=0.02, beta1=0.99, epsilon=1e-1)

style_weight=1e-2
content_weight=1e4

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss
  
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image)) 
  
train_step(image)
train_step(image)
train_step(image)
plt.imshow(image.read_value()[0])

# New Section

In [0]:
import imageio
import numpy as np

# Construct 16-bit gradient greyscale image
im = np.mgrid[0:65536:1].reshape(256,256)

# Save as PNG with imageio
imageio.imwrite('result.jpg',im)


################################

#TO READ HDR IMAGES WITHOUT LOSS
#import cv2
#import numpy as np

#filename = "GoldenGate_2k.hdr"
#im = cv2.imread(filename, cv2.IMREAD_ANYDEPTH)

################################

In [0]:
from google.colab import drive
drive.mount('/content/drive')